## 초기 환경 세팅

- 패키지 설치

- 경로 설정

- 폴더 구조

```
    BASE_DIR
    ├ data              // 초기에 제공되는 데이터들
    │ ├ private         // 교수님께서 제공한 private 테스트 이미지 (png 파일 15,000장)
    │ │ │ 00000.png
    │ │ │ ...
    │ │ └ 14999.png
    │ ├ public          // 교수님께서 제공한 public 테스트 이미지 (png 파일 10,000장)
    │ │ │ 00000.png
    │ │ │ ...
    │ │ └ 09999.png
    │ ├ fashion-mnist_test.csv      // kaggle에서 다운로드 받은 Fashion-MNIST 테스트 데이터
    │ ├ fashion-mnist_train.csv     // kaggle에서 다운로드 받은 Fashion-MNIST 학습 데이터
    │ ├ public_label.txt            // 교수님께서 제공한 public result
    │ ├ t10k-images-idx3-ubyte
    │ ├ t10k-labels-idx1-ubyte
    │ ├ train-images-idx3-ubyte
    │ └ train-labels-idx1-ubyte
    ├ model
    │ └ raw_svm.pkl
    ├ result
    │ └ raw_svm_testResult.txt
    ├ baseCode.ipynb
    ├ KNN.ipynb
    ├ LDA&Ensemble.ipynb
    ├ PCA.ipynb
    └ README.md
```

In [3]:
# 필요한 패키지 설치
# !pip install pandas numpy matplotlib scikit-learn tqdm seaborn opencv-python scikit-image xgboost

In [4]:
import os

# 데이터셋 파일 경로 설정
BASE_DIR = os.getcwd()
DATA_DIR = BASE_DIR + "/data"
MODEL_DIR = MODEL_DIR = BASE_DIR + '/model'
RESULT_DIR = BASE_DIR + '/result'

In [5]:
import csv
import tqdm
import pandas as pd
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from typing import Literal
import cv2

def load_data(kind:Literal['train', 'test', 'public', 'private']) -> tuple[np.ndarray, pd.DataFrame]:
    """
        kind: 불러올 데이터의 종류.\n
    """
    header, labels, imgs = list(), list(), list()
    header = [f'pixel{idx+1}' for idx in range(784)]
    
    # Fashion MNIST 파일 불러오기
    if kind in ['train', 'test']:
        DATA_PATH = DATA_DIR + f"/fashion-mnist_{kind}.csv"
        with open(DATA_PATH, 'r') as data:
            for idx, row in tqdm.tqdm(enumerate(csv.reader(data))):
                if idx==0:
                    header = row[1:]
                else:
                    label, img = row[0], row[1:]
                    labels.append(label)
                    imgs.append(img)
    
    # public 파일 불러오기
    elif kind=='public':
        FILE_PATH = f'{DATA_DIR}/public'
        FILE_LIST = sorted(os.listdir(f'{FILE_PATH}'))
        for filename in tqdm.tqdm(FILE_LIST):
            img = cv2.imread(f'{FILE_PATH}/{filename}', cv2.IMREAD_GRAYSCALE).flatten()
            imgs.append(img)
        with open(f'{DATA_DIR}/public_label.txt', 'r') as f:
            for line in f.readlines():
                labels.append(line.strip().split(' ')[1])
    
    # private 파일 불러오기
    elif kind=='private':
        FILE_PATH = f'{DATA_DIR}/private'
        FILE_LIST = sorted(os.listdir(f'{FILE_PATH}'))
        for filename in tqdm.tqdm(FILE_LIST):
            img = cv2.imread(f'{FILE_PATH}/{filename}', cv2.IMREAD_GRAYSCALE).flatten()
            imgs.append(img)
    

    labels = np.array(labels, dtype='int32')
    imgs = pd.DataFrame(imgs, columns=header, dtype='float32')
    return labels, imgs




In [ ]:
train_labels, train_imgs = load_data('train')
test_labels, test_imgs = load_data('test')
public_labels, public_imgs = load_data('public')
private_labels, private_imgs = load_data('private')